In [ ]:
from soview import *
import pandas as pd

In [ ]:
seqfish_path = 'zhiyuanyuan/PUBDT/st/seqfish'
h5ad_path = f'{seqfish_path}/hippo'

In [ ]:
seqfish_raw_file1 = f'{seqfish_path}/hippo_raw/NIHMS820662-supplement-7.xlsx'

In [ ]:
df_count = pd.read_excel(seqfish_raw_file1,sheet_name=0,header=None)
df_fov = pd.read_excel(seqfish_raw_file1,sheet_name=1,header=None)
df_spatial = pd.read_excel(seqfish_raw_file1,sheet_name=2,header=None)

In [ ]:
df_count

In [ ]:
adata = ad.AnnData(X = df_count.values[:,1:].astype('int').transpose())
adata.var_names = [i[1:-1] for i in df_count.values[:,0]]

In [ ]:
adata.obs['fov'] = df_fov.values[0,:]

In [ ]:
df_spatial.values[:,:2]

In [ ]:
adata.obsm['spatial_3d'] = df_spatial.values
adata.obsm['spatial'] = df_spatial.values[:,:2]

In [ ]:
for fov in np.unique(adata.obs['fov']):
    cur_ad = adata[adata.obs['fov']==fov]
    cur_ad.write_h5ad(f'{h5ad_path}/hippo_fov{fov}.h5ad')

In [ ]:
from soview import *
import shutil
import os
import gc

root_path = 'zhiyuanyuan/projects/SODB/'
source_h5ad_path = h5ad_path
biotech_category = 'Spatial_Transcriptomics'
biotech = 'seqFISH'
# 多个单词要加下划线
dataset_short = 'Shah2016InSitu'

h5ad_list = os.listdir(source_h5ad_path)
out_dataname_list = []
out_h5ad_path_list = []
out_feature_dict_list = []
N_Unit_list = []
N_Gene_list = []
for hl in h5ad_list:
    if not hl.endswith('.h5ad'):
        continue
    # source_h5ad_data = hl.split('.')[0]
    source_h5ad_data = hl[0:-5]


    source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'

    data_name = source_h5ad_data
    
    import os

    path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
    path_abs = root_path+path_rel
    os.makedirs(path_abs, exist_ok=False)
    print("The new directory is created!")
    shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')
    
    
    data_use = path_rel
    adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
    print(adata.X)
    print(f'h5ad path: {data_use}/data.h5ad')
    
    adata.obsm['spatial'] = np.array(adata.obsm['spatial'][:,[0,1]])
    
    adata.layers["counts"] = adata.X.copy()
    
    # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
    if True:
        # is all the element in adata.X are integer?
    
        sc.pp.normalize_per_cell(adata)
        sc.pp.log1p(adata)
    # if adata.shape[1]>=2000:
    #     sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
    # else:
    #     pass
    

    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata)
    
    sc.pl.embedding(adata,basis='spatial',color='leiden')
    sc.tl.umap(adata,n_components=3)
    SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
    print(f'View path: {data_use}/View_UMAP_LAB.pdf')
    
    gene_list = np.array(adata.var_names).tolist()
    obs_list = np.array(adata.obs.keys()).tolist()
    obsm_list = []
    for obsm in np.array(adata.obsm.keys()).tolist():
        for i in range(adata.obsm[obsm].shape[1]):
            cur_obsm = f'{obsm}@{i}'
            obsm_list.append(cur_obsm)
            
    feature_dict = {
        'gene':gene_list,
        'obs':obs_list,
        'obsm':obsm_list
    }
    
    import pickle
    with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
        pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    
    adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
    out_dataname_list.append(source_h5ad_data)
    out_h5ad_path_list.append(f'{data_use}/data.h5ad')
    out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
    N_Unit_list.append(adata.shape[0])
    N_Gene_list.append(adata.shape[1])
    
    print(source_h5ad_data)
    print(f'h5ad path: {data_use}/data.h5ad')
    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    print(f'N_Unit: {adata.shape[0]}')
    print(f'N_Gene: {adata.shape[1]}')
    del adata
    gc.collect()